In [1]:
# unifica_saeb.py
import pandas as pd
from glob import glob
import unicodedata
import re
import sys

# -------- CONFIGURAÇÃO (ajuste se quiser) --------
FILE_PATTERN = "SAEB_UNIFICADO_*.csv"  # padrão para localizar seus arquivos
SEP = ';'                              # separador informado por você
ENC = 'latin-1'                        # encoding informado por você
PROF_COL_STANDARD = "PROFICIENCIA_MT_SAEB"  # nome que vamos usar internamente
OUTFILE = "SAEB_MT_UNIFICADO.csv"   # arquivo de saída
# -------------------------------------------------

def normalize(name):
    """Normaliza um nome de coluna: remove acentos, pontuação e espaços, retorna minúsculo."""
    if not isinstance(name, str):
        return ""
    s = name.strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r'[^a-z0-9]+', '_', s)
    s = s.strip('_')
    return s

def find_prof_col(cols):
    """
    Tenta identificar a coluna de proficiência entre uma lista de colunas.
    Estratégia:
      1) match exato sobre o nome normalizado do padrão
      2) checar presença simultânea dos tokens 'proficiencia','mt','saeb'
      3) fallback: qualquer coluna que contenha 'proficiencia'
    Retorna o nome original da coluna se encontrada, ou None.
    """
    normalized_target = normalize(PROF_COL_STANDARD)
    # 1) match exato
    for c in cols:
        if normalize(c) == normalized_target:
            return c
    # 2) tokens principais
    tokens = ['proficiencia', 'mt', 'saeb']
    for c in cols:
        nc = normalize(c)
        if all(tok in nc for tok in tokens):
            return c
    # 3) fallback por 'proficiencia'
    for c in cols:
        if 'proficiencia' in normalize(c):
            return c
    return None

def read_csv_safe(path):
    """Lê CSV com SEP e ENC definidos; retorna DataFrame ou lança erro detalhado."""
    try:
        df = pd.read_csv(path, sep=SEP, encoding=ENC)
        return df
    except Exception as e:
        raise RuntimeError(f"Erro lendo '{path}' com sep='{SEP}' e encoding='{ENC}': {e}")

def classificacao1(score):
    """Intervalos contínuos para valores decimais."""
    if pd.isna(score):
        return pd.NA
    s = float(score)
    if s < 225:
        return 'Insuficiente'
    elif 225 <= s < 300:
        return 'Básico'
    elif 300 <= s < 350:
        return 'Proficiente'
    elif s >= 350:
        return 'Avançado'
    return pd.NA

def classificacao2(c1):
    """Agrupa Básico+Insuficiente => Insuficiente; Proficiente+Avançado => Proficiente"""
    if pd.isna(c1):
        return pd.NA
    if c1 in ['Insuficiente', 'Básico']:
        return 'Insuficiente'
    if c1 in ['Proficiente', 'Avançado']:
        return 'Proficiente'
    return pd.NA

def main():
    files = glob(FILE_PATTERN)
    if not files:
        print(f"Nenhum arquivo encontrado com padrão '{FILE_PATTERN}'. Ajuste FILE_PATTERN ou coloque os CSVs na mesma pasta.")
        sys.exit(1)

    dfs = []
    arquivos_sem_prof = []
    for f in files:
        print(f"Lendo: {f} ...")
        df = read_csv_safe(f)
        # limpa espaços nas colunas (mantém o nome original)
        df.columns = [c.strip() if isinstance(c, str) else c for c in df.columns]
        prof_col = find_prof_col(df.columns)
        if prof_col is None:
            arquivos_sem_prof.append(f)
            dfs.append(df)  # inclui mesmo assim; terá NaNs na coluna padrão
        else:
            # renomeia a coluna detectada para o nome padrão uniforme
            if prof_col != PROF_COL_STANDARD:
                df = df.rename(columns={prof_col: PROF_COL_STANDARD})
            dfs.append(df)

    if arquivos_sem_prof:
        msg = ("Atenção: os seguintes arquivos NÃO tiveram a coluna de proficiência detectada "
               f"(baseada em '{PROF_COL_STANDARD}'):\n" + "\n".join(arquivos_sem_prof) +
               "\n\nEles foram incluídos, mas a coluna de proficiência ficará com NaN nesses casos. "
               "Se isso for inesperado, verifique os cabeçalhos desses arquivos.")
        print(msg)

    # união das colunas (ordem por primeira aparição)
    all_cols = []
    for df in dfs:
        for c in df.columns:
            if c not in all_cols:
                all_cols.append(c)

    # Se a coluna padrão não apareceu em nenhum arquivo, informar e abortar
    if PROF_COL_STANDARD not in all_cols:
        print(f"ERRO: Nenhuma coluna detectada foi renomeada para '{PROF_COL_STANDARD}'. "
              "Verifique os cabeçalhos dos arquivos ou ajuste a função de detecção.")
        sys.exit(1)

    # reindexa (preenche NaN onde coluna ausente) e concatena
    dfs = [df.reindex(columns=all_cols) for df in dfs]
    df_all = pd.concat(dfs, ignore_index=True)

    # converte proficiência para numérico
    df_all[PROF_COL_STANDARD] = pd.to_numeric(df_all.get(PROF_COL_STANDARD), errors='coerce')

    # aplica classificações
    df_all['classificação1'] = df_all[PROF_COL_STANDARD].apply(classificacao1)
    df_all['classificação2'] = df_all['classificação1'].apply(classificacao2)

    # salva em utf-8-sig (bom para Excel)
    df_all.to_csv(OUTFILE, index=False, encoding='utf-8-sig')

    # resumo
    print("\nUnificação concluída.")
    print("Arquivo salvo em:", OUTFILE)
    print("Arquivos processados:", len(files))
    print("Linhas totais no unificado:", len(df_all))
    print("\nContagem por classificação1:")
    print(df_all['classificação1'].value_counts(dropna=False))
    print("\nContagem por classificação2:")
    print(df_all['classificação2'].value_counts(dropna=False))

if __name__ == "__main__":
    main()


Lendo: SAEB_UNIFICADO_2019.csv ...
Lendo: SAEB_UNIFICADO_2017.csv ...
Lendo: SAEB_UNIFICADO_2021.csv ...
Lendo: SAEB_UNIFICADO_2023.csv ...

Unificação concluída.
Arquivo salvo em: SAEB_MT_UNIFICADO.csv
Arquivos processados: 4
Linhas totais no unificado: 7585929

Contagem por classificação1:
classificação1
Básico          4048241
Insuficiente    2247109
Proficiente     1123206
Avançado         167373
Name: count, dtype: int64

Contagem por classificação2:
classificação2
Insuficiente    6295350
Proficiente     1290579
Name: count, dtype: int64
